# trainer

> Trains the schedule
- order: 7

In [1]:
#| default_exp trainer

In [2]:
!which python

/Users/kobus/aishiftscheduler/.venv/bin/python


In [3]:
# Set up autoreload
%load_ext autoreload
%autoreload 2

In [4]:
#| hide
from nbdev.showdoc import *

In [5]:
#| export
# from collections import namedtuple, defaultdict
import numpy as np
import pandas as pd
# import matplotlib as mpl
# import matplotlib.pyplot as plt
from copy import copy
import time
# import math
# from pprint import pprint
## !pip install -U "ray"
import ray
# import json

import aishiftscheduler.config as cf
import aishiftscheduler.model as mod
import aishiftscheduler.policy as pol

/Users/kobus/aishiftscheduler/aishiftscheduler/shift_scheduler_data.xlsx
/Users/kobus/aishiftscheduler/aishiftscheduler/AIShiftScheduler1.png

len(thNAMES)=5
thNAMES=['thCumSlots', 'thSickProb', 'thCumMerits', 'thContSlots', 'thSelect']


In [6]:
pd.__version__

'2.0.3'

In [9]:
#| export
pd.options.display.float_format = '{:,.4f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [11]:
#| export
def setup_thetas_for_training():
    # Setup the training thetas.
    ThetasCumSlots = np.arange(cf.TH_CumSlots_SPEC[0], cf.TH_CumSlots_SPEC[1], cf.TH_CumSlots_SPEC[2])
    ## ThetasCumSlots = TH_CumSlots_SPEC
    
    ## ThetasSickProb = np.arange(TH_SickProb_SPEC[0], TH_SickProb_SPEC[1], TH_SickProb_SPEC[2])
    ThetasSickProb = cf.TH_SickProb_SPEC
    
    ThetasCumMerits = np.arange(cf.TH_CumMerits_SPEC[0], cf.TH_CumMerits_SPEC[1], cf.TH_CumMerits_SPEC[2])
    ## ThetasCumMerits = TH_CumMerits_SPEC
    
    ## BALANCE/REMAINING 
    ## ThetasContSlots = np.arange(TH_ContSlots_SPEC[0], TH_ContSlots_SPEC[1], TH_ContSlots_SPEC[2])
    ## ThetasContSlots = TH_ContSlots_SPEC
    
    ThetasSelect = cf.TH_Select_SPEC
    
    thetas_list = [
        ( round(th0,2), round(th1,2), round(th2,2), round(1-(th0+th1+th2),2), th4 )
        for th0 in ThetasCumSlots
          for th1 in ThetasSickProb
            for th2 in ThetasCumMerits if (th0 + th1 + th2) <= 1.0
              for th4 in ThetasSelect
    ]
    Thetas = [
        ## P.build_theta({'thCumSlots': tup[0], 'thSickProb': tup[1], 'thCumMerits': tup[2], 'thSelect': tup[3]})
        pol.Policy(None).build_theta({
          'thCumSlots': tup[0], 
          'thSickProb': tup[1], 
          'thCumMerits': tup[2], 
          'thContSlots': tup[3], 
          'thSelect': tup[4]
          })
        for tup in thetas_list
    ]
    return Thetas

In [12]:
#| export
def do_train(L, T, Thetas, pars):
  M = mod.Model(pars)
  P = pol.Policy(M)
  ## DEM = DemandSimulator(seed=SEED_TRAIN)
  ## MER = MeritSimulator(seed=SEED_TRAIN)

  return \
    P.parallel_perform_grid_search_sample_paths('X__Alloc', Thetas, L, T, pars)

In [13]:
#| export
def train_schedule(L, T, First_n_t, Last_n_t, pars):
    start = time.time()
    ray.shutdown()
    ray.init()
    Thetas = setup_thetas_for_training()

    ThetaStar_expCbarcum, ThetaStar_expCtilcum, \
    Cbarcum, Ctilcum, \
    Best_theta, Worst_theta, \
    Best_Cbarcum, Worst_Cbarcum, \
    Best_Ctilcum, Worst_Ctilcum, \
    Record = \
      do_train(L, T, Thetas, pars)
    
    f'{ThetaStar_expCbarcum.iloc[-1]=:.2f}'
    Df_first_n_t = pd.DataFrame.from_records(Record[:First_n_t], columns=pars.LABELS)
    Df_last_n_t = pd.DataFrame.from_records(Record[-Last_n_t:], columns=pars.LABELS)
    end = time.time(); print(f'EXECUTION TIME: {end - start} seconds')
    return Best_theta, Worst_theta, Df_first_n_t, Df_last_n_t

In [14]:
#| hide
import nbdev; nbdev.nbdev_export()